In [ ]:
# Find most relevant terms for each topic using LDA clustering

In [ ]:
import pandas as pd
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

In [ ]:
df_transcripts = pd.read_csv("transcripts_m10.csv")

In [ ]:
#df_transcripts.head(5)

In [ ]:
tfv = TfidfVectorizer(ngram_range = (1,1))

In [ ]:
vec_text = tfv.fit_transform(df_transcripts['lemmatized_text'])

In [ ]:
words = tfv.get_feature_names()

In [ ]:
words[:10]

In [ ]:
# now working through https://medium.com/@yanlinc/how-to-build-a-lda-topic-model-using-from-text-601cdcbfd3a6

In [ ]:
lda_model = LatentDirichletAllocation(n_components=10)

#https://www.kaggle.com/rajmehra03/topic-modelling-using-lda-and-lsa-in-sklearn
lda_output = lda_model.fit_transform(vec_text)

In [ ]:
print(lda_output)  # Model attributes

In [ ]:
df_documents = pd.DataFrame(lda_output)

In [ ]:
words = tfv.get_feature_names()

In [ ]:
max(lda_model.components_[0])

In [ ]:
for i, comp in enumerate(lda_model.components_):
    words_comp = dict(zip(words, comp))
    sorted_words = sorted(words_comp.items(), reverse=True, key=lambda item: item[1])
    print("Topic", i)
    for w in sorted_words[:10]:
        print(w[0], w[1])
    print("\n")

In [ ]:
# make a prediction# make a prediction
vec_text = ["people many think like working know getting rich",
           "giving devane lectures",
           "might saying china vietnam interested called reform"]
lda_model.fit_transform(tfv.transform(vec_text))

In [ ]:
# view top document matches for a particular category

In [ ]:
df_all = pd.concat([df_documents, df_transcripts], axis=1)
df_all.sort_values(2, ascending=False).head(20)